In [247]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [111]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

## RL results

In [129]:
df1 = pd.read_csv('/projects/rlmolecule/pstjohn/tempo/20210707_rl_dft_spin_bv_run_21.csv.gz', index_col=0)
df2 = pd.read_csv('/projects/rlmolecule/pstjohn/tempo/20210719_rl_dft_spin_bv_run_22.csv.gz', index_col=0)
df3 = pd.read_csv('/projects/rlmolecule/pstjohn/tempo/20220319_rl_dft_spin_bv_run_23.csv.gz', index_col=0)

df = pd.concat([df1, df2, df3])
df.head()

,smiles,atom_type,atom_index,spin,buried_vol
0,C=C(C)[C]1C(C)=CC=C1C,C,0,0.334919,38.391431
1,C=C(C)[C]1C(C)=CC=C1C,C,1,-0.130092,50.158327
2,C=C(C)[C]1C(C)=CC=C1C,C,2,0.015376,38.276312
3,C=C(C)[C]1C(C)=CC=C1C,C,3,0.569167,60.527549
4,C=C(C)[C]1C(C)=CC=C1C,C,4,-0.091715,50.450485


In [166]:
len(df.smiles.unique())

1019

In [178]:
df.to_csv('/projects/rlmolecule/pstjohn/20211020_paper_supplement/data/rl_candidates_spin_bv.csv.gz', index=False)

In [176]:
mols_ml = pd.read_csv('/projects/rlmolecule/pstjohn/tempo/fp_all_ml_hits.csv.gz')

In [182]:
pd.read_csv('/projects/rlmolecule/pstjohn/tempo/20220320_rl_dft_redox.csv').head()

,smiles,max_spin_ml,spin_buried_vol_ml,ionization_energy_ml,electron_affinity_ml,stability_ml,max_spin_dft,spin_buried_vol_dft,stability_dft,ionization_energy_dft,electron_affinity_dft
0,C/C=C(CC)\C(C#N)=C/[CH]S,0.234689,57.867332,0.926315,-0.319963,96.132891,0.420341,40.450792,69.433753,0.702990,-0.379919
1,COc1cc([C@H](C)OC)sc1[O],0.244859,57.371970,0.997670,-0.199483,95.129038,0.282597,58.613235,94.483394,0.891072,-0.340028
2,C=C(C(C)=S)/C(C#N)=C(\C)[CH]O,0.312372,61.988106,0.770350,-0.393657,96.369491,0.241585,40.253455,78.174193,0.685521,-0.603238
3,C=C(CCO)/C(C#N)=C\[CH]S,0.231923,57.446888,0.996620,-0.224065,95.850732,0.338014,51.824523,84.923803,0.854011,-0.332545
4,CCc1c[nH]c([O])c1C(S)C=O,0.306346,64.174740,0.872176,-0.336902,98.857428,0.281189,56.599884,92.540422,0.907861,-0.292763


## Get spin and buried volume data for new training set molecules

In [135]:
run_20 = pd.read_csv('/projects/rlmolecule/pstjohn/tempo/20220324_rl_dft_spin_bv_run_20_23.csv.gz', index_col=0)
run_20 = run_20[~run_20.smiles.isin(df3.smiles)]

In [105]:
new_data = pd.read_csv('/projects/rlmolecule/pstjohn/spin_gnn/20210211_fixed_rl_spin_bv_data.csv.gz')
spin_bv = pd.read_csv('/projects/rlmolecule/pstjohn/tempo/20220324_dft_ml_spin_bv_data_of_20210109_with_spin.csv.gz')
# new_data = new_data.rename(columns={'fractional_spin': 'spin'})[['smiles', 'atom_index', 'spin', 'buried_vol']]

In [152]:
merged_new = new_data[['err_smiles', 'smiles', 'atom_index']].merge(
    spin_bv[['smiles', 'atom_index', 'spin', 'buried_vol', 'fractional_spin']],\
    left_on=['err_smiles', 'atom_index'],
    right_on=['smiles', 'atom_index'],
    suffixes=['', '_y'], how='left').drop(['err_smiles', 'smiles_y'], axis=1)

In [153]:
import rdkit.Chem

def get_symbol(row):
    mol = rdkit.Chem.MolFromSmiles(row.smiles)
    mol = rdkit.Chem.AddHs(mol)
    return mol.GetAtomWithIdx(row.atom_index).GetSymbol()

In [154]:
merged_new['atom_type'] = merged_new.progress_apply(get_symbol, axis=1)

  0%|          | 0/54293 [00:00<?, ?it/s]

In [155]:
merged_new = merged_new.append(run_20[['smiles', 'atom_type', 'atom_index', 'spin', 'buried_vol', 'fractional_spin']])

In [156]:
merged_new.duplicated(subset=['smiles', 'atom_index']).any()

False

In [159]:
merged_new[['smiles', 'atom_type', 'atom_index', 'spin', 'buried_vol', 'fractional_spin']].to_csv(
    '/projects/rlmolecule/pstjohn/20211020_paper_supplement/data/high_scoring_spins_and_bv.csv.gz', index=False)

In [160]:
redox_new_calcs = pd.read_csv('/projects/rlmolecule/pstjohn/spin_gnn/20210216_fixed_rl_redox_data.csv')
tempo_results = pd.read_csv('/projects/rlmolecule/pstjohn/spin_gnn/tempo_results.csv')
redox_new_calcs = redox_new_calcs.append(tempo_results)

In [161]:
spin_bv_smiles = pd.Series(merged_new.smiles.unique())
redox_smiles = pd.Series(redox_new_calcs.smiles.unique())

In [145]:
redox_new_calcs.drop('err_smiles', axis=1).rename(
    columns={'ionization energy': 'oxidation',
             'electron affinity': 'reduction'}
)[['smiles', 'oxidation', 'reduction']].to_csv(
    '/projects/rlmolecule/pstjohn/20211020_paper_supplement/data/high_scoring_redox_potentials.csv.gz', index=False)

In [146]:
pd.read_csv('/projects/rlmolecule/pstjohn/20211020_paper_supplement/data/radical_db_redox_potentials.csv.gz').head()

,smiles,oxidation,reduction
0,C#C/C(=C/C=O)[C](C)C,NaN,-0.059920
1,C#C/C(=C\[CH2])CC,NaN,-0.943918
2,C#C/C(C)=C(/[CH2])C,0.781466,-0.915292
3,C#C/C(C)=C(\C)C[CH]C,0.313277,-1.811318
4,C#C/C(C)=C(\C)[C@@H]([CH2])C,NaN,-1.526529


In [179]:
pd.read_csv('/projects/rlmolecule/pstjohn/20211020_paper_supplement/data/high_scoring_redox_potentials.csv.gz').smiles.nunique()

3941

In [195]:
spins = pd.read_csv('/projects/rlmolecule/pstjohn/20211020_paper_supplement/data/high_scoring_spins_and_bv.csv.gz')

In [204]:
def stability(df):
    return 50 * (1 - df.fractional_spin.max()) + df.buried_vol[df.fractional_spin.idxmax()]

stability = spins.groupby('smiles').apply(stability)

In [205]:
stability.max()

109.71244992655048

98.4328365790613

In [194]:
test.smiles.apply(lambda x: rdkit.Chem.MolFromSmiles(x).GetNumAtoms()).min()

9

In [187]:
high_scoring_smiles

3978

In [148]:
pd.read_csv('/projects/rlmolecule/pstjohn/20211020_paper_supplement/data/radical_db_spins_and_bv.csv.gz').head()

,smiles,atom_type,atom_index,spin,buried_vol,fractional_spin
0,[CH2]C1=COCC1,C,0,0.586646,30.291542,0.405000
1,[CH2]C1=COCC1,C,1,-0.180922,41.879991,0.124902
2,[CH2]C1=COCC1,C,2,0.570409,38.252099,0.393790
3,[CH2]C1=COCC1,O,3,0.096694,33.700549,0.066754
4,[CH2]C1=COCC1,C,4,-0.002564,34.074425,0.001770


In [181]:
pd.read_csv('/projects/rlmolecule/pstjohn/20211020_paper_supplement/data/high_scoring_spins_and_bv.csv.gz').head()

,smiles,atom_type,atom_index,spin,buried_vol,fractional_spin
0,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O,C,0,0.103887,40.057887,0.085155
1,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O,C,1,-0.029665,54.431655,0.024316
2,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O,C,2,0.011449,41.549793,0.009385
3,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O,O,3,-0.001445,32.768445,0.001184
4,C=C(CO)C1=C([CH]O)C(=O)C=C(C)C1=O,C,4,0.421383,62.511166,0.345404


In [126]:
len(df.smiles.unique())

1019

In [162]:
redox_smiles[~redox_smiles.isin(spin_bv_smiles)]

Series([], dtype: object)

In [209]:
passing = pd.read_csv('/projects/rlmolecule/pstjohn/tempo/20220329_passing_dft.csv')

In [243]:
import rdkit.Chem
from rdkit import DataStructs

from scipy.spatial.distance import pdist, squareform
import numpy as np

In [244]:
mols = passing.smiles.apply(rdkit.Chem.MolFromSmiles)

def fingerprint(mol):
    return np.asarray(rdkit.Chem.RDKFingerprint(mol).ToList(), dtype=bool)

fps = np.vstack(mols.apply(fingerprint).values)

In [249]:
from sklearn.cluster import AgglomerativeClustering

In [270]:
clust = AgglomerativeClustering(n_clusters=None, affinity='precomputed', linkage='single', distance_threshold=0.3)

In [272]:
clusters = clust.fit_predict(squareform(pdist(fps, metric='rogerstanimoto')))
len(set(clusters))

13

In [273]:
passing['cluster'] = clust.fit_predict(squareform(pdist(fps, metric='rogerstanimoto')))

In [285]:
passing

,smiles,max_spin_ml,spin_buried_vol_ml,ionization_energy_ml,electron_affinity_ml,stability_ml,max_spin_dft,spin_buried_vol_dft,stability_dft,ionization_energy_dft,electron_affinity_dft,V_diff,pass_redox,cluster
0,COc1cc([C@H](C)OC)sc1[O],0.244859,57.371970,0.997670,-0.199483,95.129038,0.282597,58.613235,94.483394,0.891072,-0.340028,1.231100,True,0
1,CCc1c[nH]c([O])c1C(S)C=O,0.306346,64.174740,0.872176,-0.336902,98.857428,0.281189,56.599884,92.540422,0.907861,-0.292763,1.200624,True,1
2,Cc1sc([O])c([C@H](C)S)c1C,0.293660,62.598217,0.949479,-0.258639,97.915202,0.255540,53.833899,91.056880,1.047752,-0.254123,1.301875,True,0
3,C=C(C)C1=C(SC)[CH]C=C1,0.241431,58.222950,1.049057,-0.184555,96.151410,0.230548,55.591280,94.063870,0.868759,-0.192382,1.061141,True,5
4,CS[C@@H](C)c1c(CN)c[nH]c1[O],0.317376,66.194470,0.733074,-0.400398,100.325667,0.075347,44.661544,90.894211,0.718637,-0.436463,1.155100,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,CNC/C=C/[C]1C(C)=CC=C1C,0.217957,56.395145,0.844885,-0.384302,95.497300,0.287403,55.828434,91.458263,0.930745,-0.301008,1.231753,True,5
228,CC1=CC=C(C)[C]1/C=C/CCO,0.189184,56.585010,0.890140,-0.385841,97.125825,0.284523,55.918445,91.692290,0.945357,-0.356518,1.301875,True,5
229,CC1=CC=C(C)[C]1C1(O)CCC1,0.273460,65.447470,1.061694,-0.270874,101.774452,0.463972,65.554371,92.355765,0.853249,-0.176219,1.029467,True,5
230,CC1=CCC([CH]C#N)=C1C=NO,0.213656,55.684708,0.888831,-0.351111,95.001928,0.212200,55.172099,94.562088,0.909412,-0.352817,1.262229,True,5


In [284]:
passing[~mols.apply(lambda x: 
                   x.HasSubstructMatch(rdkit.Chem.MolFromSmarts('[s;R]c[O!R]')) or 
                   x.HasSubstructMatch(rdkit.Chem.MolFromSmarts('[s;R]cc[O!R]')
                                      ))]

,smiles,max_spin_ml,spin_buried_vol_ml,ionization_energy_ml,electron_affinity_ml,stability_ml,max_spin_dft,spin_buried_vol_dft,stability_dft,ionization_energy_dft,electron_affinity_dft,V_diff,pass_redox,cluster
1,CCc1c[nH]c([O])c1C(S)C=O,0.306346,64.174740,0.872176,-0.336902,98.857428,0.281189,56.599884,92.540422,0.907861,-0.292763,1.200624,True,1
3,C=C(C)C1=C(SC)[CH]C=C1,0.241431,58.222950,1.049057,-0.184555,96.151410,0.230548,55.591280,94.063870,0.868759,-0.192382,1.061141,True,5
4,CS[C@@H](C)c1c(CN)c[nH]c1[O],0.317376,66.194470,0.733074,-0.400398,100.325667,0.075347,44.661544,90.894211,0.718637,-0.436463,1.155100,True,1
6,CS[C@@H](CN)c1cc(C)oc1[O],0.262447,59.321415,0.787642,-0.324902,96.199051,0.318122,58.198382,92.292300,0.884568,-0.338722,1.223290,True,7
12,C/C=C(CCO)\C(C#N)=C/[CH]S,0.256951,59.462498,0.926629,-0.299904,96.614961,0.213085,61.672296,101.018030,0.988568,-0.399511,1.388079,True,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,CNC/C=C/[C]1C(C)=CC=C1C,0.217957,56.395145,0.844885,-0.384302,95.497300,0.287403,55.828434,91.458263,0.930745,-0.301008,1.231753,True,5
228,CC1=CC=C(C)[C]1/C=C/CCO,0.189184,56.585010,0.890140,-0.385841,97.125825,0.284523,55.918445,91.692290,0.945357,-0.356518,1.301875,True,5
229,CC1=CC=C(C)[C]1C1(O)CCC1,0.273460,65.447470,1.061694,-0.270874,101.774452,0.463972,65.554371,92.355765,0.853249,-0.176219,1.029467,True,5
230,CC1=CCC([CH]C#N)=C1C=NO,0.213656,55.684708,0.888831,-0.351111,95.001928,0.212200,55.172099,94.562088,0.909412,-0.352817,1.262229,True,5


In [274]:
passing.groupby('cluster').sample(1)

,smiles,max_spin_ml,spin_buried_vol_ml,ionization_energy_ml,electron_affinity_ml,stability_ml,max_spin_dft,spin_buried_vol_dft,stability_dft,ionization_energy_dft,electron_affinity_dft,V_diff,pass_redox,cluster
162,CC(C)(CCO)c1ccc([O])s1,0.325022,64.534380,1.071847,-0.117864,98.283284,0.338199,66.117948,99.207982,1.001711,-0.197770,1.199481,True,0
98,CCSCc1c(C=S)c[nH]c1[O],0.302919,60.458370,0.964129,-0.185709,95.312441,0.365939,59.835998,91.539045,1.010391,-0.060682,1.071073,True,1
222,Cc1c([O])sc2c1SCC2(C)C,0.324155,64.039970,0.996482,-0.148305,97.832245,0.407462,64.135663,93.762540,0.983507,-0.273797,1.257304,True,2
231,Cc1cc([O])oc1[C@H](C)C(C)C,0.303757,62.589390,0.886705,-0.270195,97.401560,0.398364,61.347027,91.428816,0.869439,-0.368518,1.237957,True,3
167,C/C=C(\CC)c1scc(N)c1[O],0.189926,59.823730,0.898619,-0.209622,100.327435,0.298878,58.120501,93.176623,0.878147,-0.156056,1.034202,True,4
178,CC/C=C\C1=C(C(C)C)C=C[CH]1,0.249742,59.832603,0.941568,-0.328292,97.345488,0.286509,58.066874,93.741441,1.025276,-0.241144,1.266419,True,5
20,CCC(C)(C)c1c(S)coc1[O],0.315457,69.621720,0.830588,-0.256599,103.848890,0.341007,69.068361,102.018014,1.190609,-0.123838,1.314446,True,6
6,CS[C@@H](CN)c1cc(C)oc1[O],0.262447,59.321415,0.787642,-0.324902,96.199051,0.318122,58.198382,92.292300,0.884568,-0.338722,1.223290,True,7
132,C[C@H](N)c1c([O])sc2ccsc12,0.256954,61.199093,1.050393,-0.036820,98.351395,0.287297,60.286178,95.921331,0.970691,-0.045933,1.016624,True,8
60,C/C=C1/CCc2csc([O])c21,0.257423,58.943546,0.965410,-0.181220,96.072382,0.020804,47.631063,96.590858,1.172350,-0.045688,1.218038,True,9


In [286]:
data = pd.read_csv('/projects/rlmolecule/pstjohn/tempo/20220320_rl_dft_redox.csv')

In [287]:
data

,smiles,max_spin_ml,spin_buried_vol_ml,ionization_energy_ml,electron_affinity_ml,stability_ml,max_spin_dft,spin_buried_vol_dft,stability_dft,ionization_energy_dft,electron_affinity_dft
0,C/C=C(CC)\C(C#N)=C/[CH]S,0.234689,57.867332,0.926315,-0.319963,96.132891,0.420341,40.450792,69.433753,0.702990,-0.379919
1,COc1cc([C@H](C)OC)sc1[O],0.244859,57.371970,0.997670,-0.199483,95.129038,0.282597,58.613235,94.483394,0.891072,-0.340028
2,C=C(C(C)=S)/C(C#N)=C(\C)[CH]O,0.312372,61.988106,0.770350,-0.393657,96.369491,0.241585,40.253455,78.174193,0.685521,-0.603238
3,C=C(CCO)/C(C#N)=C\[CH]S,0.231923,57.446888,0.996620,-0.224065,95.850732,0.338014,51.824523,84.923803,0.854011,-0.332545
4,CCc1c[nH]c([O])c1C(S)C=O,0.306346,64.174740,0.872176,-0.336902,98.857428,0.281189,56.599884,92.540422,0.907861,-0.292763
...,...,...,...,...,...,...,...,...,...,...,...
1073,CC/C(=C/[CH]C=O)C(CC)CC,0.318210,66.462020,0.863992,-0.357325,100.551500,0.283909,31.496519,67.301074,0.920650,-0.249688
1074,C=C([C]1C(C)=CC=C1C)N(C)N,0.284826,63.943413,0.848473,-0.309218,99.702103,0.257853,37.869587,74.976947,NaN,NaN
1075,COC1([C]2C(C)=CC=C2C)CC1,0.324443,67.059440,0.994485,-0.232017,100.837285,0.404906,52.427147,82.181861,NaN,-0.347647
1076,CC[C@H](/C(C)=C\[CH]C=O)C(C)C,0.329344,61.961792,0.865190,-0.270451,95.494575,0.290277,46.019289,81.505414,NaN,-0.512028
